<a href="https://colab.research.google.com/github/hidt4/python-compchem-book/blob/main/compchem_book_ch04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 4章 分子をコンピュータで扱う方法について知ろう

### 環境構築

#### Google Colab上にPsi4をインストール

In [1]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y psi4=1.8 python=3.10 -c conda-forge/label/libint_dev -c conda-forge
import sys
sys.path.append('/usr/local/lib/python3.10/site-packages')

--2024-06-05 12:28:07--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.176.84, 104.18.177.84, 2606:4700::6812:b054, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.176.84|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2024-06-05 12:28:07--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143808873 (137M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 137.15M   207MB/s    in 0.7s    

2024-06-05 12:28:07 (207 MB/s) - ‘Minicond

In [2]:
import os
import datetime
import numpy as np
import pandas as pd
import psi4

print(f'current time: {datetime.datetime.now()}')
print(f'python version:\n{sys.version}')
print(f'numpy version: {np.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'psi4 version: {psi4.__version__}')

current time: 2024-06-05 12:32:58.653272
python version:
3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
numpy version: 1.25.2
pandas version: 2.0.3
psi4 version: 1.8.2


#### 計算資源の設定

In [ ]:
# 計算資源の確認（CPU, RAM）
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 8
core id		: 0
cpu cores	: 4
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
addres

In [ ]:
!cat /proc/meminfo

MemTotal:       53474568 kB
MemFree:        41133904 kB
MemAvailable:   51973024 kB
Buffers:          209352 kB
Cached:         10834048 kB
SwapCached:            0 kB
Active:          1198880 kB
Inactive:       10403032 kB
Active(anon):        952 kB
Inactive(anon):   558824 kB
Active(file):    1197928 kB
Inactive(file):  9844208 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:            186688 kB
Writeback:             0 kB
AnonPages:        556352 kB
Mapped:           449556 kB
Shmem:              1264 kB
KReclaimable:     425744 kB
Slab:             505228 kB
SReclaimable:     425744 kB
SUnreclaim:        79484 kB
KernelStack:        6400 kB
PageTables:        11924 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    26737284 kB
Committed_AS:    2987596 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       10336 kB
VmallocChunk:          0 kB
Percpu:          

In [ ]:
n_cpu = os.cpu_count()

In [ ]:
ram = os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES') / (1024 ** 3)

In [ ]:
# 環境に応じて計算資源を設定
psi4.set_num_threads(n_cpu)
psi4.set_memory(f'{ram * 0.9: .0f}GB')

46000000000

### Psi4のMoleculeオブジェクトについて知ろう

In [ ]:
# Moleculeオブジェクトの作成
h2o = psi4.geometry('''
0 1
O       -0.1176269719      0.7387773605      0.0000000000
H        0.8523730281      0.7387773605      0.0000000000
H       -0.4409567836      1.4770262439     -0.5397651517
''')

In [ ]:
type(h2o)

psi4.core.Molecule

#### オブジェクトの複製

In [ ]:
h2o_2 = h2o.clone()

In [ ]:
h2o == h2o_2

False

In [ ]:
print(h2o.save_string_xyz())

0 1
 O    0.000000000000   -0.000000000000   -0.062675830412
 H    0.792000605218    0.000000000000    0.497355455604
 H   -0.792000605218   -0.000000000000    0.497355455604



In [ ]:
print(h2o_2.save_string_xyz())

0 1
 O    0.000000000000   -0.000000000000   -0.062675830412
 H    0.792000605218    0.000000000000    0.497355455604
 H   -0.792000605218   -0.000000000000    0.497355455604



#### 代表的なメソッドの使い方

In [ ]:
print(f'molecular charge: {h2o.molecular_charge()}')
print(f'multiplicity: {h2o.multiplicity()}')
print(f'number of atoms: {h2o.natom()}')

molecular charge: 0
multiplicity: 1
number of atoms: 3


In [ ]:
bohr2ang = psi4.constants.bohr2angstroms

In [ ]:
for i in range(h2o.natom()):
    x = h2o.x(i) * bohr2ang
    y = h2o.y(i) * bohr2ang
    z = h2o.z(i) * bohr2ang

    print(f'{h2o.symbol(i)}\t{x:.3f}\t{y:.3f}\t{z:.3f}')

O	0.000	-0.000	-0.063
H	0.792	0.000	0.497
H	-0.792	-0.000	0.497


In [ ]:
print(h2o.save_string_xyz())

0 1
 O    0.000000000000   -0.000000000000   -0.062675830412
 H    0.792000605218    0.000000000000    0.497355455604
 H   -0.792000605218   -0.000000000000    0.497355455604



In [ ]:
print(h2o.save_string_xyz_file())

3

 O    0.000000000000   -0.000000000000   -0.062675830412
 H    0.792000605218    0.000000000000    0.497355455604
 H   -0.792000605218   -0.000000000000    0.497355455604



### py3Dmolで分子を描画しよう

In [ ]:
!pip install py3Dmol
import py3Dmol

In [ ]:
def show_3D(mol: psi4.core.Molecule) -> py3Dmol.view:
    """
    Psi4のMoleculeオブジェクトを描画する
    Args:
        mol: 描画対象の分子
    Return:
        py3Dmol.view: py3Dmolの描画オブジェクト

    """
    view = py3Dmol.view(width=400, height=400)
    xyz = mol.save_string_xyz_file()
    view.addModel(xyz, 'xyz')
    view.setStyle({'stick': {}})
    view.setBackgroundColor('#e1e1e1')
    view.zoomTo()

    return view.show()

In [ ]:
show_3D(h2o)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol